In [1]:
import pandas as pd
import sed_eval
from tqdm import tqdm_notebook as tqdm

In [3]:
import imp
milsed = imp.load_package('milsed', '../../milsed/')

Using TensorFlow backend.


In [8]:
prepare = imp.load_source('prepare', '../../models/01_prepare.py')

In [10]:
test_idx = pd.read_json('../../models/resources/index_test.json')

In [11]:
labelfile = '/home/bmcfee/data/groundtruth_strong_label_testing_set.csv'

In [44]:
def score_null(idx):
    segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(prepare.DCASE_CLASSES,
                                                                     time_resolution=1.0)
    
    for fid in tqdm(idx['id']):
        
        ref_jam = milsed.utils.create_dcase_jam(fid,
                                                labelfile,
                                                duration=10.0,
                                                weak=False)
        
        ann_r = ref_jam.annotations.search(annotation_tools='reference')[0]
        
        ref_list = []
        for event in ann_r.data:
            ref_list.append({'event_onset': event.time,
                             'event_offset': event.time + event.duration,
                             'event_label': event.value})
        ref_list = sed_eval.util.event_list.EventList(ref_list)

        est_list = sed_eval.util.event_list.EventList([])

        segment_based_metrics.evaluate(ref_list, est_list)

    return segment_based_metrics

In [45]:
score = score_null(test_idx)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [46]:
r = score.results()

/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/sed_eval/sound_event.py:481: RuntimeWarning: Mean of empty slice
  'f_measure': float(numpy.nanmean(event_wise_f_measure)),
/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/sed_eval/sound_event.py:482: RuntimeWarning: Mean of empty slice
  'precision': float(numpy.nanmean(event_wise_precision)),


In [47]:
r['overall']

{'accuracy': {'accuracy': 0.9402723362283898,
  'balanced_accuracy': 0.5,
  'sensitivity': 0.0,
  'specificity': 1.0},
 'error_rate': {'deletion_rate': 1.0,
  'error_rate': 1.0,
  'insertion_rate': 0.0,
  'substitution_rate': 0.0},
 'f_measure': {'f_measure': nan, 'precision': nan, 'recall': 0.0}}